<!--bibtex

@article{Villard1995,
   author = {Gilles Villard},
   journal = {J. Symbolic Comput.},
   pages = {269-286},
   title = {Generalized subresultants for computing the Smith normal form of polynomial matrices.},
   volume = {20},
   year = {1995},
}

@article{HongHoughKogan,
   author = {Hoon Hong and Zachary Hough and Irina A Kogan},
   keywords = {polynomial vectors,rational curves MSC 2010: 12Y05, 13P10, 14Q05, 68W30,syzygy module,µ-basis},
   title = {Algorithm for computing µ-bases of univariate polynomials},
}
-->

### $\mu$-Basis 

$\textbf{deg}$ function will give the highest degree of a variable "s" in a vector "a"  

In [2]:
def deg(vector,s):
    
    higest_degree=0
    components=[]
    
    for i in range(0,len(vector)):
        components.append(SR(vector[i]))
        if higest_degree<components[i].degree(s):
            higest_degree=components[i].degree(s)
            
    return higest_degree

$\textbf{degmatrix}$ function will give the highest degree of a variable "s" in a matrix.

In [3]:
def degmatrix(matrix,s):
    
    higest_degree=0
    
    for i in range(0,matrix.ncols()):
        for j in range(0,matrix.nrows()):
            if higest_degree<SR(matrix[j,i]).degree(s):
                higest_degree=SR(matrix[j,i]).degree(s)
                
    return higest_degree

$\textbf{GCDlist}$ function will return the GCD of all elements of a list.

In [4]:
def GCDlist(L):
    
    n=len(L)
    if n==1:
        gcd_list=L[0]
    elif n==2:
        gcd_list=gcd(L[0],L[1])
    else:
        gcd_list=gcd(L[0],L[1])
        for i in range(2,n):
            gcd_list=gcd(gcd_list,L[i])
            
    return gcd_list

$\textbf{GCD_matrix}$ function will return the GCD of all elements of a matrix.

In [6]:
def GCD_matrix(matrix):
    n=matrix.nrows()
    m=matrix.ncols()
    gcd_matrix=0
    
    for i in range(0,n):
        list_of_columns=[matrix[i,j] for j in range(0,m)]
        gcd_i=GCDlist(list_of_columns)
                
        gcd_matrix=gcd(gcd_i,gcd_matrix)
    return gcd_matrix

The following function $\textbf{block}$ takes a matrix and an order and returns a block matrix of the specific order. 

In [7]:
def block(M,n):
    
    m=M.nrows()
    
    for i in range(0,n-m):
        M=block_matrix([[1,zero_matrix(1,M.ncols())],[zero_matrix(M.ncols(),1),M]],subdivide=False)
        
    return M

### Univariate Case (degree of t=0)

Let $a=(a_{10}(s), a_{20}(s), a_{30}(s), a_{40}(s))$

<ol>
   <li> Construct a matrix from b with coefficients of s in $a=(a_{10}(s), a_{20}(s), a_{30}(s), a_{40}(s))$ with coefficients of degree 0 in first row, degree 1 in second row and so on. If n is number of elements of a and m is the highest  degree of s in $a$, the b will be a $(m+1)\times n$ matrix.<br />
For example if $a=(1+s^2+s^4, 1+s^3+s^4, 1+s^4)$,
\[matrix\text{_}coefficients=\begin{pmatrix}
1&1&1\\
0&0&0\\
1&0&0\\
0&1&0\\
1&1&1
\end{pmatrix}\]</li>
    
<li> Then construct a block matrix, matrix_final with matrix_coeffficients in its diagonal and zero as rest of the entries. It will be of size $(2m+1) \times (m+1)n$.
</li>
</ol>
Following function $\textbf{Matrix}$ will give the resulting matrix after inputing a univariate vector and the variable (here the variable is taken as s)

In [8]:
def Matrix(row_s,s):
    
    n=len(row_s)
    degree_s=deg(row_s,s)
    
    coefficient_list=[]
    
    for j in range(0,degree_s+1):
        for i in range(0,n):
            coefficient_list.append((SR(row_s[i])).coefficient(s,j))

    matrix_coefficients=matrix(degree_s+1,n,coefficient_list)

    matrix_block=[]
    
    for i in range(0,degree_s+1):
        matrix_single_block=block_matrix([[zero_matrix(QQ,i,n)],[matrix_coefficients],[zero_matrix(QQ,degree_s-i,n)]],subdivide=False)
        matrix_block=matrix_block+matrix_single_block.columns()
        
    matrix_final=(matrix(n*(degree_s+1),(2*degree_s)+1,matrix_block))
    
    return matrix_final.transpose()

We follow the method in :cite:t:`HongHoughKogan` to compute $\mu-$basis of univariate polynomials. 
<ol>
<li> Construct the partial reduced row-echelon form of the matrix matrix_final following these steps.

<ul>
    <li> Stop the forward elimination as soon as $n-1$ basic non-pivot columns are  detected.</li>
    <li> Skip over periodic non-pivot columns.</li>
    <li> Carry out the row operations only on the required columns.</li>
</ul>
</li>
<li> Construct a matrix whose columns form a $\mu$-basis for $a$.
</li>
    We will get a matrix $M(s)$ such that \[(a_{10}(s) , a_{20}(s) , a_{30}(s) , a_{40}(s))\cdot M(s)=( 0 , 0 , 0).\]
</ol>
The following $\textbf{GaussJordan}$ returns the $\mu$-basis $M(s)$ and the partial reduced row-echelon form of the matrix

.. bibliography::

In [9]:

def GaussJordan(M,n):
    
    pivot=list(M.pivots())
    non_pivot=M.nonpivots()
    
    equivalence_class=[]
    basic_non_pivot=[] 
    periodic_non_pivot=[]
    non_pivot_column_actual=[] 
    
    for i in range(0,len(non_pivot)):
        m=mod(non_pivot[i]+1,n)
        if m in equivalence_class:
            periodic_non_pivot.append(non_pivot[i])
        else:
            equivalence_class.append(m)
            basic_non_pivot.append(non_pivot[i])
            non_pivot_column_actual.append(non_pivot[i]+1)
            
    pivot_column=[] 
    pivot_column_actual=[] 
    
    for i in pivot:
        if i<basic_non_pivot[len(basic_non_pivot)-1]:
            pivot_column.append(i)
            pivot_column_actual.append(i+1)
            
            
    selected_columns=sorted(pivot_column+basic_non_pivot)
    matrix_selected_columns=M.matrix_from_columns(selected_columns)
    echelon_matrix=matrix_selected_columns.echelon_form()
    
    partial_echelon=[]
    count=0

    for i in range(0,M.ncols()):
        if i in selected_columns:
            partial_echelon.append(echelon_matrix.column(count))
            count=count+1
        else:
            partial_echelon.append(M.column(i))
            
    
    partial_echelon_matrix=(matrix(M.ncols(),M.nrows(),partial_echelon)).transpose()
    var('s')
    R1.<s>=QQ[]
    mu_basis=zero_matrix(R1,n,n-1)
    
    
    A=[]
    for j in range(0,n-1):
        quotient,reminder=divmod(non_pivot_column_actual[j]-1,n) 
        mu_basis[reminder,j]=mu_basis[reminder,j]+s^quotient
        
    for i in range(0,len(pivot_column_actual)): 
        quotient,reminder=divmod(pivot_column_actual[i]-1,n)
        for j in range(0,n-1):
            mu_basis[reminder,j]=mu_basis[reminder,j]-(partial_echelon_matrix[i,non_pivot_column_actual[j]-1]*s^quotient)
            
    return mu_basis, partial_echelon_matrix


The following function $\textbf{mu_basis_zero}$ will give the $\mu-$ basis in a univariate case, that is when degree of t=0

In [10]:
def mu_basis_zero(P):
    
    matrix_P=Matrix(P,s)
    mu_basis=GaussJordan(matrix_P,len(P))
    
    return mu_basis[0]

### Degree of t=1

Let $\mathcal{P}=\{a_1(s,t),a_2(s,t),a_3(s,t),a_4(s,t)\}$, where
\begin{equation}\label{quad}
a_i(s,t)=a_{i0}(s)+a_{i1}(s)t,\, i=1,2,3,4,
\end{equation} 



<ol>
    <li> Find the degree of $\mathcal{P}$ in s which is the highest degree of s in the elements of $\mathcal{P}$.</li>
    <li> Extract the matrix $A(s)$ from $\mathcal{P}$ and the first row of $A(s)$.
    $$A(s):=\begin{pmatrix}
    a_{10}(s) & a_{20}(s) & a_{30}(s) & a_{40}(s)\\
    a_{11}(s) & a_{21}(s) & a_{31}(s) & a_{41}(s)
    \end{pmatrix}\in\mathbb{K}[s]^{2\times4}.
    $$
    and 
    \[a=(a_{10}(s), a_{20}(s), a_{30}(s), a_{40}(s))\]</li>
    
   </ol>
   
The following function $\textbf{vector_to_matrix}$ will return thr matrix $A(s)$

In [11]:

def vector_to_matrix(vector):
    
    var('s,t')
    R1.<s>=QQ[]
    
    n=len(vector)
    m=deg(vector,t)
    A=zero_matrix(R1,m+1,n)
    
    for i in range(0,n):
        for j in range(0,m+1):
            A[j,i]=A[j,i]+vector[i].coefficient(t,j)
    
    return A

Find the matrix $M'(s)$ such that $$(a_{10}(s) , a_{20}(s) , a_{30}(s) , a_{40}(s))\cdot M'(s)=( 0 , 0 , 0)$$ as in the univariate case


Choose now a B\'ezout identity $$b_1(s)a_{10}(s)+\ldots +b_4(s)a_{40}(s)=g(s)$$ with $\deg(b_i)\leq m,\ i=1,2,3,4.$ 
Given by the following function $\textbf{Bezout_coefficient}$ 

In [12]:
def Bezout_coefficient(a):
    
    coefficient=0
    coefficient_bezout=[]
    gcd_list=[]
    R2.<s>=QQ[]
    n=len(a)
    
    for i in range(1,n):
        if len(gcd_list)==0:
            gcd_i,coefficient_1,coefficient_2=xgcd(R2(a[i-1]),R2(a[i]))
            gcd_list.append(gcd_i)
            gcd_bezout=gcd_i
            coefficient=coefficient_1
            coefficient_bezout.append(coefficient_2)
        else:
            gcd_i,coefficient_1_i,coefficient_2_i=xgcd(gcd_list[i-2],R2(a[i]))
            gcd_list.append(gcd_i)
            gcd_bezout=gcd_i
            coefficient=coefficient_1_i*coefficient
            
            for j in range(0,len(coefficient_bezout)):
                coefficient_bezout[j]=coefficient_1_i*coefficient_bezout[j]
            coefficient_bezout.append(coefficient_2_i)
            
    coefficient_bezout.insert(0,coefficient)
    
    return matrix(n,1,coefficient_bezout), gcd_bezout

Now set $M_1(s)$ to have as its first column the vector $(b_1(s) \ b_2(s)\ b_3(s) b_4(s)),$ and the $\mu$-basis obtained in previous step ($M'(s)$) as the remaining $3$ columns so that 
$$A_1(s):=A(s)\cdot M_1(s)=\begin{pmatrix}
g(s)& 0& 0 & 0\\
a'_1(s)& a'_2(s)& a'_3(s)& a'_4(s)
\end{pmatrix}.
$$

Now repeat the above steps to find $\mu-$basis of  $(a'_2(s), a'_3(s), a'_4(s))$ and contruct the matrix $M_2(s)$ so that 
$$M(s):=M_1(s)\cdot \begin{pmatrix} 1&{\bf0}\\ {\bf0} & M_2(s)\end{pmatrix}$$ 
(Do this by using the function $\textbf{block}$)

Then 
$$A(s)\cdot M(s)=\begin{pmatrix}
g(s)&0&0&0\\
a'_1(s)&g'(s)&0&0
\end{pmatrix}$$

Set $g''(s):=\gcd(g(s)+a'_1(s)t, g'(s)t),$
and $\alpha(s,t):=\frac{g(s)+a'_1(s)t}{g''(s)}, \beta(s,t):=\frac{g'(s)}{g''(s)}.$


A $\mu$-basis of $P(s,t)$ is given by $-\beta(s,t)M^1(s)+\alpha(s,t)M^2(s),\, M^3(s), M^4(s)$ where $M^1(s) \ M^2(s) \ M^3(s) \ M^4(s)$ are the columns of $M(s)$.

The following function $\textbf{mu_basis_one}$ will give the $\mu-$ basis when degree of t=1

In [13]:
def mu_basis_one(P):
    A=vector_to_matrix(P)
    d=degmatrix(A,s)
    matrix_reduce=A
    reduction_matrix=identity_matrix(len(P))
    
    for i in range(0,A.nrows()):
        a=matrix_reduce[0]
        if len(a)!=1:
            m1=Matrix(a,s)
            mu_basis_a=GaussJordan(m1,len(a)) 
            bezout_coeff_a=Bezout_coefficient(a) 
            reduced_intermediate=block_matrix([[bezout_coeff_a[0],mu_basis_a[0]]],subdivide=False) 
            m5=block(reduced_intermediate,len(P))
            reduction_matrix=reduction_matrix*m5
            m2=matrix_reduce*reduced_intermediate
            matrix_reduce=m2[1:m2.nrows(),1:m2.ncols()]
        else: break
            
    reduced_matrix=A*reduction_matrix
    L=list([reduced_matrix[0,0]+(reduced_matrix[1,0]*t),reduced_matrix[1,1]*t])
    g=GCDlist(L)
    alpha=(reduced_matrix[0,0]+(reduced_matrix[1,0]*t))/g
    beta=reduced_matrix[1,1]*t/g
    mu=zero_matrix(SR,len(P),len(P)-1)
    
    for i in range(0,reduction_matrix.ncols()-1):
        if i==0:
            mu[:,i]=mu[:,i]+(-beta*reduction_matrix[:,i]+alpha*reduction_matrix[:,i+1])
        else:
            mu[:,i]=mu[:,i]+(reduction_matrix[:,i+1])
            
    return mu,reduced_matrix

## Degree of t=2

Let $\mathcal{P}=\{a_1(s,t),a_2(s,t),a_3(s,t),a_4(s,t)\}$, where
\begin{equation}
a_i(s,t)=a_{i0}(s)+a_{i1}(s)t+a_{i2}t^2,\, i=1,2,3,4,
\end{equation}

Extract the matrix $A(s)$ from $\mathcal{P}$ using the  $\textbf{vector_to_matrix}$ function.

$$A(s):=\begin{pmatrix}
a_{10}(s) & a_{20}(s) & a_{30}(s) & a_{40}(s)\\
a_{11}(s) & a_{21}(s) & a_{31}(s) & a_{41}(s)\\
a_{12}(s) & a_{22}(s) & a_{32}(s) & a_{42}(s)
\end{pmatrix}\in\mathbb{K}[s]^{3\times4}$$
so that 
$$(1,t,t^2)\cdot A(s)=\mathcal{P}$$



#### Good Conditioning


$A$ good conditioning on $A$ ($n\times n$ matrix) is an (n-1)-tuple $(\alpha_2, \alpha_3,..., \alpha_n)$ such that if the first row of A, $A_1$ is replaced by $A_1+\alpha_2A_2+...+\alpha_nA_n$. Then the gcd of elements of first row is equal to the gcd of all elements of the matrix.

<ol>
    <li> Let $A$ be an $n\times n$ matrix</li>
    <li> Find the degree of $A$ in s which is the highest degree of s in the elements of $A$. Let it be d.</li>
    <li> Let $f=\{1,2,3,...,2d+1\}$, subset of field with $2d+1$ elements. ( We are assuming the base field has ore than $2d+1$ elements.)</li>
    <li> Let g be the gcd of elements of first row of $A$ and $g_i$ be the gcd of elements of i-th row of A. While $g_1 \neq g_i$, add a multiple (taken from f) of i-th row of $A$ to the first row. The elements of f for which $g_1 = g_i$ will be the $\alpha_i$ s.</li> 
    <li> Return the modified matrix. It will be a good conditioning of $A$.</li>
</ol>

The following $\textbf{good_conditioning}$ will return the modified matrix, say $A_1$ and matrix of good conditining, say $U$ such that $$U\cdot A=A_1$$

In [16]:
def good_conditioning(A,multiple_possible):
    
    row_multiple=[0]*(A.nrows()-1)
    elements_column_1=[A[0,j] for j in range(0,A.ncols())]
    
    gcd_row_1=GCDlist(elements_column_1)
    gcd_A=GCD_matrix(A)
    
    elements_A=[]
    for i in range(1,A.nrows()):
        elements_A=elements_A+[A[i,j]for j in range(0,A.ncols())]
        
    for i in range(1,A.nrows()):
        
        k=0
        if gcd_row_1!=gcd_A:
            
            elements_new_row=[A[0,j]+ multiple_possible[k]*A[i,j] for j in range(0,A.ncols())]
            gcd_row_1=GCDlist(elements_new_row)
            
            gcd_A=GCDlist(elements_new_row + elements_A)
            row_multiple[i-1]=multiple_possible[k]

            k=k+1
            
        else: break
            
    matrix_good_conditioning=(identity_matrix(A.nrows()))
    
    for i in range(1,A.nrows()):
        matrix_good_conditioning=matrix_good_conditioning.with_added_multiple_of_row(0,i,row_multiple[i-1])
        
    new_matrix=matrix_good_conditioning*A
    
    return new_matrix, matrix_good_conditioning

Follow as in the case of degree of t =1 on three rows of $A$ while doing good conditioning at each step to obtain 

\begin{equation}\label{rhs}
U_A\cdot A(s)\cdot M(s)=A_1(s)\cdot M(s)=\begin{pmatrix}
1&0&0&0\\
a_{11}'(s)&g'(s)&0&0\\
a''_{12}(s)& g'(s)a^*_{22}(s) &g'(s) g''(s) & 0
\end{pmatrix}
\end{equation}

where $U_A$ is the product of matrices of good conditioning obtained at each row of $A$.

The good conditioning is done at each step so that the element $g'(s)$ will be the gcd of all elements below and towards its right. 

The following function $\textbf{reduce}$ will take the parametriization $P$ as input and give the matrices $U, A(s), M(s)$ and $N(s)$ so that
$$U\cdot A(s)\cdot M(s)=N(s)$$
where, N(s) will have the form
$$N(s)=\begin{pmatrix}
1&0&0&0\\
a_{11}'(s)&g'(s)&0&0\\
a''_{12}(s)& g'(s)a^*_{22}(s) &g'(s) g''(s) & 0
\end{pmatrix}$$

In [17]:
def reduce(P):
    
    matrix_P=vector_to_matrix(P)
    matrix_to_reduce=matrix_P
    
    reduction_matrix=identity_matrix(len(P))
    final_good_conditioning=identity_matrix(matrix_P.nrows())
    
    for i in range(0,matrix_P.nrows()):
        
        d=degmatrix(matrix_to_reduce,s)
        f=(list(range(1,2*d+2)))
        matrix_good_conditioned,good_conditioning_inter=good_conditioning(matrix_to_reduce,f)
        
        block_good_conditioning=block(good_conditioning_inter,matrix_P.nrows())
        final_good_conditioning= block_good_conditioning*final_good_conditioning
        
        a=(good_conditioning_inter*matrix_to_reduce)[0]
        
        if len(a)!=1 and a!=vector([0]*len(a)):
            
            matrix_row_a=Matrix(a,s)
            mu_basis_a=GaussJordan(matrix_row_a,len(a)) 
            bezout_coeff_a=Bezout_coefficient(a) 
            
            reduced_matrix_1=block_matrix([[bezout_coeff_a[0],mu_basis_a[0]]],subdivide=False) 
            reduction_row_a=block(reduced_matrix_1,len(P))
            reduction_matrix=reduction_matrix*reduction_row_a
            reduced_row_a=matrix_good_conditioned*reduced_matrix_1
            
            matrix_to_reduce=reduced_row_a[1:reduced_row_a.nrows(),1:reduced_row_a.ncols()]
            
        else: break
            
    reduced_matrix=final_good_conditioning*matrix_P*reduction_matrix
    
    final_reduced=reduced_matrix/(reduced_matrix[0,0])
    matrix_P_final=matrix_P/(reduced_matrix[0,0])
    
    return final_good_conditioning, matrix_P_final, reduction_matrix, final_reduced

If determinant of $N(s)$ is zero, then this will be reduced to the case of degree of t is one and the $\mu-$ basis can be computed similarly. 

The following function $\textbf{mu_det_zero}$ will give the $\mu-$basis in such a case.

In [18]:
def mu_det_zero(P):
    
    matrix_good_conditioning, matrix_P,matrix_to_multiply,reduced_matrix=reduce(P)
    Phi=vector([1,t,t^2])*(matrix_good_conditioning.inverse())

    
    P2=Phi*reduced_matrix[:,:2]
    mu=zero_matrix(SR,reduced_matrix.ncols(),len(P)-1)
    for i in range(0,len(P)-1):
        if i==0:
            for j in range(0,reduced_matrix.ncols()):
                mu[j,i]=((matrix_to_multiply.column(0))*P2[1]-(P2[0]*matrix_to_multiply.column(1)))[j]
        else:
            for j in range(0,reduced_matrix.ncols()):
                mu[j,i]=(matrix_to_multiply.column(i+1))[j]
    return mu


Let $$\phi=(\phi_1,\phi_2,\phi_3):=(1,t,t^2)\cdot U_A^{-1}$$

$$\mathcal{P}\cdot M(s)=(1,t,t^2)\cdot U_A^{-1}\cdot U_A\cdot A(s)\cdot M(s)$$

$$\mathcal{P}\cdot M(s)=\phi \cdot \begin{pmatrix}
1&0&0&0\\
a_{11}'(s)&g'(s)&0&0\\
a''_{12}(s)& g'(s)a^*_{22}(s) &g'(s) g''(s) & 0
\end{pmatrix}$$

Thus $\mathcal{P}\cdot M(s)$ will be of the form,
$$(\alpha,g'(s) \beta, g'(s)\gamma, 0)$$



 If $g'(s)\neq 1$, then follow the same steps as above for 
 $$\mathcal{P}'=(\alpha,\beta, \gamma)$$
 
 Let $B(s)$ be the new matrix and $B_1(s)=U_B\cdot B(s)$ after good conditioning such that 
 \begin{equation}\label{g2}
B_1(s)\cdot M'(s)=\begin{pmatrix}
1&0&0\\
b_{11}'(s)&1&0\\
b''_{12}(s)& b^*_{22}(s) & g''(s) 
\end{pmatrix}
\end{equation}

Thus we will have:
   \begin{equation}
U\cdot A(s)\cdot M(s)=\begin{pmatrix}
1&0&0&0\\
a_{11}'(s)&1&0&0\\
a''_{12}(s)& a^*_{22}(s) & g(s) & 0
\end{pmatrix}
\end{equation}
where $U$ is the matrix of good conditioning of $A(s)$. 


Multiplying $M(s)$ to the right by 
$$\begin{pmatrix} 
1&0 &0&0\\
-a'_{11}(s) &1 &0&0\\
0& 0&1&0\\
0&0&0&1
\end{pmatrix}$$
to obtain 

\begin{equation}
U\cdot A(s)\cdot M''(s)=\begin{pmatrix}
1&0&0&0\\
0&1&0&0\\
a''_{12}(s)& a^*_{22}(s) & g(s) & 0
\end{pmatrix},
\end{equation}



We have 
$$(1,t,t^2)\cdot A(s)=\mathcal{P}$$
(Note that this will be $\mathcal{P}'$ for $g(s)\neq 1$)

and let $$\phi=(\phi_1,\phi_2,\phi_3):=(1,t,t^2)\cdot U^{-1}$$

where $U$ is the matrix of good conditioning of $A(s)$ such that $U\cdot A(s)=A_1(s)$.

$$\mathcal{P}\cdot M''(s)=(1,t,t^2)\cdot U^{-1}\cdot U\cdot A(s)\cdot M''(s)=\phi\cdot A_1(s)\cdot M''(s)$$
    
  $$\phi\cdot A_1(s)\cdot M''(s)=\phi\cdot \begin{pmatrix}
1&0&0&0\\
0&1&0&0\\
a''_{12}(s)& a^*_{22}(s) & g(s) & 0
\end{pmatrix}$$


Let $$\alpha(t):=\begin{pmatrix}
\alpha_{11}(t) & \alpha_{12}(t) & \alpha_{13}(t)\\
\alpha_{21}(t) & \alpha_{22}(t) & \alpha_{23}(t)
\end{pmatrix} =\begin{pmatrix}
\alpha_1(t)\\
\alpha_2(t)\end{pmatrix}
\in\mathbb{K}[t]^{2\times3}
$$

be the $\mu$-basis of $(\phi_1,\phi_2,\phi_3)$. Compute it with function \textbf{GaussJordan} as this is a univariate case in variable $t$.

$\sum_{i=1}^4 h_i(s,t)M^i(s) \in \mbox{Syz}(P(s,t))$ if and only if   $A(s)\cdot M(s)\begin{pmatrix} h_1(s,t)\\ h_2(s,t)\\ h_3(s,t)\\ h_4(s,t)\end{pmatrix}$ is a $\mathbb{K}[s,t]$-linear combination of $(\alpha_1(t),\,\alpha_2(t))$.\\

Removing the fourth column of $M(s)$ as it already gives a zero column, we have

$$
\begin{pmatrix}
1&0&0\\
0&1&0\\
a''_{12}(s)& a^*_{22}(s) & g(s) 
\end{pmatrix}\cdot \begin{pmatrix} h_1(s,t)\\ h_2(s,t)\\ h_3(s,t)\end{pmatrix}= \big( \alpha_1(t) \
\alpha_2(t)\big)\cdot \begin{pmatrix}
\nu_1(s,t)\\
\nu_2(s,t)
\end{pmatrix},
$$


with  $\nu_1(s,t), \nu_2(s,t)\in\mathbb{K}[s,t].$ We multiply this equality with the inverse of the matrix in the left-hand-side to get


\begin{equation}
\begin{pmatrix} h_1(s,t)\\ h_2(s,t)\\ h_3(s,t)\end{pmatrix} = 
\begin{pmatrix}
1&0&0\\
0&1&0\\
-\frac{a''_{12}(s)}{g(s)}& -\frac{a^*_{22}(s)}{g(s)} & \frac{1}{g(s)} 
\end{pmatrix}\cdot 
\big( \alpha_1(t) \
\alpha_2(t)\big)\cdot \begin{pmatrix}
\nu_1(s,t)\\
\nu_2(s,t)
\end{pmatrix},
\end{equation}

\begin{equation}\label{hh}
    \begin{pmatrix} h_1(s,t)\\ h_2(s,t)\\ h_3(s,t)\end{pmatrix} = 
\begin{pmatrix}
\alpha_{11}(t)&\alpha_{21}(t)\\
\alpha_{12}(t)&\alpha_{22}(t)\\
\mathcal{D}_1/g(s)
& 
\mathcal{D}_2/g(s)
\end{pmatrix}\cdot 
 \begin{pmatrix}
\nu_1(s,t)\\
\nu_2(s,t)
\end{pmatrix}.
\end{equation}


where $\mathcal{D}_1=-a''_{12}(s)\alpha_{11}(t) -a^*_{22}(s)\alpha_{12}(t)+ \alpha_{13}(t)$ and $\mathcal{D}_2=-a''_{12}(s)\alpha_{21}(t) -a^*_{22}(s)\alpha_{22}(t)+ \alpha_{23}(t)$.


This is equivalent to requiring that 
$\mathcal{D}_1\nu_1(s,t)+ \mathcal{D}_2\nu_2(s,t)$
being a multiple of $g(s),$ i.e. there exists $\nu_3(s,t)\in\mathbb{K}[s,t]$ such that

$$ \mathcal{D}_1\nu_1(s,t)+ \mathcal{D}_2\nu_2(s,t)+g(s)\nu_3(s,t)=0.
$$


Let
$$X_i(s,t)=(x_{i1}(s,t),x_{i2}(s,t),x_{i3}(s,t)),\, i=1,2.
$$
be the $\mu-$basis of \begin{equation}
(a''_{12}(s)\alpha_{11}(t)+a^*_{22}(s)\alpha_{12}(t)-\alpha_{13}(t),
a''_{12}(s)\alpha_{21}(t)+a^*_{22}(s)\alpha_{22}(t)-\alpha_{23}(t), 
g(s))\end{equation}



Let
\begin{equation}\label{ele}
L_i(s,t):=\sum_{j=1}^3L_{ij}(s,t)M^j(s)\in \mbox{Syz}(P(s,t)), \ \ i=1,2.
\end{equation}
Then \{$L_1(s,t),\, L_2(s,t),\, M^4(s)\}$ will give the $\mu-$basis of the parametrization where $M^4(s) will be the fourth column of $M(s)$

The following function $\textbf{mu_basis_two}$ will give the $\mu-$ basis when degree of t=2

In [19]:
def mu_basis_two(P):
    matrix_good_conditioning, matrix_P,matrix_to_multiply,reduced_matrix=reduce(P)
    
    if det(reduced_matrix[:reduced_matrix.nrows(),:reduced_matrix.nrows()])==0:
        return mu_det_zero(P)
    else:
        n=reduced_matrix.nrows()
        parametrization=P
        Pm=matrix_to_multiply
        new_parametrization=[]
        value=reduced_matrix[1,1]
        if reduced_matrix[1,1]!=1:
            g=1
        else:
            g=0
        if g==1:
            for i in range(0,reduced_matrix.nrows()):
                if i==0:
                    new_parametrization.append((P*matrix_to_multiply)[i])
                else:
                    new_parametrization.append((P*matrix_to_multiply)[i]/reduced_matrix[1,1])

            matrix_good_conditioning, matrix_P,matrix_to_multiply,reduced_matrix=reduce(new_parametrization)
            parametrization=new_parametrization

        

        if det(reduced_matrix[:reduced_matrix.nrows(),:reduced_matrix.nrows()])!=0:
            
            Phi=vector([1,t,t^2])*(matrix_good_conditioning.inverse())
            matrix_to_multiply_inter=identity_matrix(SR,len(parametrization))
            matrix_to_multiply_inter[1,0]=-reduced_matrix[1,0]
            final_matrix_to_multiply=matrix_to_multiply*matrix_to_multiply_inter
            B1=(matrix_good_conditioning*matrix_P*final_matrix_to_multiply).simplify_full()
            
            #mu basis of phi
            m1=Matrix(Phi,t)
            GJ=GaussJordan(m1,len(Phi)) 
            mu_phi=GJ[0].subs(s=t)


            g11=B1[2,2]
            matrix_inverse=B1[:B1.nrows(),:B1.nrows()].inverse()
            matrix_linear_combination=matrix_inverse*mu_phi
            Param=vector([matrix_linear_combination[2,0]*g11,matrix_linear_combination[2,1]*g11,g11])
            mu_linear_combination=mu_basis_one(Param)[0]
            vector_syzygy_1=(matrix_inverse*mu_phi*mu_linear_combination[:-1,:-1]).simplify_full()
            vector_syzygy_2=(matrix_inverse*mu_phi*mu_linear_combination[:-1,1:]).simplify_full()

            mu_basis=zero_matrix(SR,len(parametrization),len(parametrization)-1)
            for i in range(0,final_matrix_to_multiply.ncols()-2):
                if i==0:
                    mu_basis[:,i]=mu_basis[:,i]+matrix([(vector_syzygy_1[0,0]*final_matrix_to_multiply.column(0))+(vector_syzygy_1[1,0]*final_matrix_to_multiply.column(1))+(vector_syzygy_1[2,0]*final_matrix_to_multiply.column(2))]).transpose()
                    mu_basis[:,i+1]=mu_basis[:,i+1]+matrix([(vector_syzygy_2[0,0]*final_matrix_to_multiply.column(0))+(vector_syzygy_2[1,0]*final_matrix_to_multiply.column(1))+(vector_syzygy_2[2,0]*final_matrix_to_multiply.column(2))]).transpose()
                else:
                    mu_basis[:,i+1]=mu_basis[:,i+1]+(final_matrix_to_multiply[:,i+2])


        elif det(reduced_matrix[:reduced_matrix.nrows(),:reduced_matrix.nrows()])==0:
            mu_basis=mu_det_zero(parametrization)
        if g==1:
            muvector_syzygy_1=mu_basis.with_rescaled_row(0,value)
            mu_basis_new=Pm[:,:n]*muvector_syzygy_1
            if (len(P)-n)!=0:
                mu_basis_new=block_matrix(SR,[[(mu_basis_new),matrix(4,1,Pm.column(-1))]], subdivide=False)
            return mu_basis_new
        elif g==0:
            return mu_basis



In [96]:
def muBasis(P):   
    if deg(P,t)==0:
        return mu_basis_zero(P)
    elif deg(P,t)==1:
        return mu_basis_one(P)[0]
    elif deg(P,t)==2:
        return mu_basis_two(P)
    else:
        print("Degree of t greater than 2. We are working on it :)")

In [21]:
import time
start_time = time.time()
var('s,t')
P=vector([4*s^3+s*t^2+4*s^2-12*s*t+t^2+s+1, 4*s^4+s^2*t^2+s^2+6*t,6*t^2, 4*s^2+t^2+1])
mu=muBasis(P)
time=(time.time() - start_time)
print("--- %s seconds ---" % time)

--- 2.5673985481262207 seconds ---


In [22]:
mu

[                                                                                                                                                   -1/13812*(160*s^3 - 44*s^2 - 63*s - 69)*(4*s^2 + 1) + 1/13812*(640*s^4 - 176*s^3 + 68*s^2 - 44*s + 1151*t - 23)*s                                                                                    1/1228800*(640*s^4 - 176*s^3 + 68*s^2 - 44*s - 1151*t - 23)*(160*s^3 - 44*s^2 - 63*s - 69) - 1/1228800*(102400*s^6 - 56320*s^5 + 3904*s^4 - 5984*s^3 - 3308*s^2 + 2024*s + 529)*s                                                                                                                                                                                                                                                                 -1/2]
[                                                                                                                                              -80/3453*s^4 + 22/3453*s^3 + 1/13812*(160*s^2 + 116*s - 587)*(4*s^2 + 1)

In [23]:
(P*mu).simplify_full()

(0, 0, 0)

In [24]:
mu

[                                                                                                                                                   -1/13812*(160*s^3 - 44*s^2 - 63*s - 69)*(4*s^2 + 1) + 1/13812*(640*s^4 - 176*s^3 + 68*s^2 - 44*s + 1151*t - 23)*s                                                                                    1/1228800*(640*s^4 - 176*s^3 + 68*s^2 - 44*s - 1151*t - 23)*(160*s^3 - 44*s^2 - 63*s - 69) - 1/1228800*(102400*s^6 - 56320*s^5 + 3904*s^4 - 5984*s^3 - 3308*s^2 + 2024*s + 529)*s                                                                                                                                                                                                                                                                 -1/2]
[                                                                                                                                              -80/3453*s^4 + 22/3453*s^3 + 1/13812*(160*s^2 + 116*s - 587)*(4*s^2 + 1)

In [25]:
deg(mu.column(0),t),deg(mu.column(1),t),deg(mu.column(2),t)

(1, 1, 0)

In [26]:
#shows that the minors are just multiples of parametrization
[((mu.minors(len(P)-1)[i])/P[len(P)-i-1]).simplify_full() for i in range(0,len(P))]

[1324801/117964800, -1324801/117964800, 1324801/117964800, -1324801/117964800]

In [27]:
import time
start_time = time.time()
var('s,t')
P1=vector([s^2+t^2,2*s*t, s^2-t^2])
mu1=muBasis(P1)
time=(time.time() - start_time)
print("--- %s seconds ---" % time)

--- 0.3375687599182129 seconds ---


In [28]:
(P1*mu1).simplify_full()

(0, 0)

In [29]:
deg(mu1.column(0),t),deg(mu1.column(1),t)

(1, 1)

In [30]:
[((mu1.minors(len(P1)-1)[i])/P1[len(P1)-i-1]).simplify_full() for i in range(0,len(P1))]

[-1/4, 1/4, -1/4]

#### Examples from "BOUNDS FOR DEGREES OF SYZYGIES OF POLYNOMIALS DEFINING A GRADE TWO IDEAL"-Teresa Cortadellas Benitez, Carlos D'Andrea, Eulalia Montoro

In [31]:
import time
start_time = time.time()
var('s,t')
P2=vector([11-4*s+3*s^2+4*t, 5-4*s+2*s^2+4*t-2*s*t+t^2, 1+3*s^2-s^3+s^2*t,7-3*s+s^2+3*t])
mu2=muBasis(P2)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)

--- 0.2807633876800537 seconds ---


In [32]:
(mu2).simplify_full()

[-1/61*(8*s^2 - 50*s + 51)*t^2 - 2/61*(48*s^2 - 79*s - 189)*t - 480/61*s + 1035/61                            4*s^2 + 1/24*(8*s^2 - 50*s + 51)*t - 259/24*s - 109/24                                                                      -1/3*s^2 + 1]
[           120/61*s^2 + 1/61*(24*s^3 - 42*s^2 - 92*s - 129)*t - 145/61*s - 170/61                                                    -s^3 + 7/4*s^2 + 23/6*s + 43/8                                                                                 0]
[          -2/61*(12*s - 37)*t^2 + 4/61*(12*s^2 - 45*s - 46)*t + 240/61*s - 770/61                                        -2*s^2 + 1/12*(12*s - 37)*t + 15/2*s + 8/3                                                                              -5/3]
[         -3/61*(12*s - 37)*t^2 + 6/61*(24*s^2 - 29*s - 62)*t + 720/61*s - 1395/61                                      -6*s^2 + 1/8*(12*s - 37)*t + 119/8*s + 35/12                                                                         s^2 - 4/3]

In [33]:
(P2*mu2).simplify_full()

(0, 0, 0)

In [34]:
deg(mu2.column(0),t),deg(mu2.column(1),t),deg(mu2.column(2),t)

(2, 1, 0)

In [35]:
[((mu2.minors(len(P2)-1)[i])/P2[len(P2)-i-1]).simplify_full() for i in range(0,len(P2))]

[-725/72, 725/72, -725/72, 725/72]

#### Some more examples

In [36]:
import time
start_time = time.time()
var('s,t')
P3=vector([s+3*s^2+t+4*s*t-t^2, -s^2+t+t^2, s+2*s^2-2*t^2, 1+s-t])
mu3=muBasis(P3)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)

--- 0.3372814655303955 seconds ---


In [37]:
(mu3).simplify_full()

[                     -1/24*(4*s + 7)*t + 1/6*t^2 + 1/24                                    -1/6*s + 1/6*t + 1/8                                                    -1/2]
[                            1/8*(4*s + 3)*t + t^2 - 1/8                                        -7/6*s + t + 1/8                                               2*s + 1/2]
[                     5/24*(4*s + 5)*t + 5/12*t^2 - 5/24                                          5/12*t + 17/24                                                 s + 1/2]
[-1/12*(4*s + 3)*t^2 - 1/12*(8*s^2 + 10*s - 1)*t + 1/6*s                     -2/3*s^2 - 1/12*(4*s + 3)*t - 5/6*s                                                       0]

In [38]:
(P3*mu3).simplify_full()

(0, 0, 0)

In [39]:
deg(mu3.column(0),t),deg(mu3.column(1),t),deg(mu3.column(2),t)

(2, 1, 0)

In [40]:
[((mu3.minors(len(P3)-1)[i])/P3[len(P3)-i-1]).simplify_full() for i in range(0,len(P3))]

[1/72, -1/72, 1/72, -1/72]

In [41]:
#radius 1 cylinder with axis (0,0,1)

import time
start_time = time.time()
var('s,t')
P4=vector([t^2 - 1, 2*t, s*t^2 + s, t^2 + 1])
mu4=muBasis(P4)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)
mu4

--- 0.1404571533203125 seconds ---


[ 1/2*t   -1/2      0]
[   1/2  1/2*t      0]
[     0      0     -1]
[-1/2*t   -1/2      s]

In [42]:
P4

(t^2 - 1, 2*t, s*t^2 + s, t^2 + 1)

In [43]:

(P4*mu4).simplify_full()

(0, 0, 0)

In [44]:
mu4

[ 1/2*t   -1/2      0]
[   1/2  1/2*t      0]
[     0      0     -1]
[-1/2*t   -1/2      s]

In [45]:
deg(mu4.column(0),t),deg(mu4.column(1),t),deg(mu4.column(2),t)

(1, 1, 0)

In [46]:
[((mu4.minors(len(P4)-1)[i])/P4[len(P4)-i-1]).simplify_full() for i in range(0,len(P4))]

[-1/4, 1/4, -1/4, 1/4]

In [47]:
import time
start_time = time.time()
var('s,t')
P5=vector([s^2+t^2+s*t,2*s*t+t^2, s^2-t^2])
mu5=muBasis(P5)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)

--- 0.15317535400390625 seconds ---


In [48]:
(P5*mu5).simplify_full()

(0, 0)

In [49]:
deg(mu5.column(0),t),deg(mu5.column(1),t)

(1, 1)

In [50]:
[((mu5.minors(len(P5)-1)[i])/P5[len(P5)-i-1]).simplify_full() for i in range(0,len(P5))]

[-1/3, 1/3, -1/3]

In [51]:
mu5.simplify_full()

[   2/3*s^2 + 1/3*(s + 2)*t + 1/3*s                     -2/3*s - 1/3*t]
[-1/3*s^2 - 1/3*(2*s + 1)*t - 2/3*s                      1/3*s + 2/3*t]
[  -2/3*s^2 - 1/3*(s - 1)*t - 1/3*s                      2/3*s + 1/3*t]

In [52]:
#g=1, len(P)=4
import time
start_time = time.time()
var('s,t')
P6=vector([s^2+t^2,2*s*t,2*t^2, s^2-t^2])
mu6=muBasis(P6)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)


--- 0.15256810188293457 seconds ---


In [53]:
mu6

[                                t                                 0                                -1]
[                 -1/2*s*t - 1/2*s                             1/2*t                                 0]
[1/2*(s*t + s)*s - 1/2*(s^2 + 1)*t           1/2*s*(t - 1) - 1/2*s*t                                 1]
[                                0                                 0                                 1]

In [54]:
(P6*mu6).simplify_full()

(0, 0, 0)

In [55]:
deg(mu6.column(0),t),deg(mu6.column(1),t),deg(mu6.column(2),t)

(1, 1, 0)

In [56]:
[((mu6.minors(len(P6)-1)[i])/P6[len(P6)-i-1]).simplify_full() for i in range(0,len(P6))]

[-1/4, 1/4, -1/4, 1/4]

In [57]:
#determinant B2 is zero
import time
start_time = time.time()
var('s,t')
P8=vector([t^2 - 1, s*t^2 + t^2, t^2 + 1])
mu8=muBasis(P8)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)
mu8

--- 0.03780245780944824 seconds ---


[-1/2*t^2 - 1/2          s + 1]
[             0             -2]
[ 1/2*t^2 - 1/2          s + 1]

In [58]:
(P8*mu8).simplify_full()

(0, 0)

In [59]:
[((mu8.minors(len(P8)-1)[i])/P8[len(P8)-i-1]).simplify_full() for i in range(0,len(P8))]

[1, -1, 1]

In [60]:
#determinant of rhs is zero
import time
start_time = time.time()
var('s,t')

P9=vector([-(s - 2)*t^2 + (2*s - 1)*t - 1, 2*s*t^2 + (s^2 + 1)*t + 1, s*t^2 + t + 1])
mu9=muBasis(P9)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)

mu9

--- 0.036871910095214844 seconds ---


[          -1/2*(t^2 - t)*s - 1/2*(s + 1)*t - 1/2                                                s]
[                                               0                                               -2]
[-1/2*(t^2 - t)*s + 1/2*(s + 1)*t + t^2 - t - 1/2                                            s + 2]

In [61]:
(P9*mu9).simplify_full()

(0, 0)

In [62]:
[((mu9.minors(len(P9)-1)[i])/P9[len(P9)-i-1]).simplify_full() for i in range(0,len(P9))]

[1, -1, 1]

##### degree 1 examples

In [63]:
import time
start_time = time.time()
var('s,t')

Pt=vector([s^3+2*s^2-s+3+t*(2*s^3+2*s^2-3*s+7),-3*s+3+t*(2*s^2-5*s+5),
         -2*s^2-2*s+3+t*(-6*s^2-8*s+4),2*s^2+s+2+t*(5*s^2+4*s+5) ])
mut=muBasis(Pt)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)

mut

--- 0.02483677864074707 seconds ---


[                             2/16875*((2*s^4 + 7*s^3 - 4*s + 31)*t + 15)*(496*s + 523) - 1/5*(s + 1)*t                                                                                          2*s + 221/421                                                                                        -2*s + 2213/842]
[-1/15*(s^2 + 3*s + 2)*(s + 1)*t + 1/16875*(496*s^2 + 1422*s + 605)*((2*s^4 + 7*s^3 - 4*s + 31)*t + 15)                                                                              s^2 + 897/421*s - 145/421                                                                         -1/2*s^2 - 78/421*s + 2081/421]
[                                   1/16875*(496*s^2 - 903*s - 903)*((2*s^4 + 7*s^3 - 4*s + 31)*t + 15)                                                                              s^2 - 1053/421*s + 72/421                                                                                  1922/421*s - 3289/842]
[                                            -2/16875*((2*s^4 + 7*s

In [64]:
(Pt*(mu_basis_one(Pt)[0])).simplify_full()

(0, 0, 0)

In [65]:
[(((mu_basis_one(Pt)[0]).minors(len(Pt)-1)[i])/Pt[len(Pt)-i-1]).simplify_full() for i in range(0,len(Pt))]

[-1/2, 1/2, -1/2, 1/2]

In [66]:
deg(mut.column(0),t),deg(mut.column(1),t),deg(mut.column(2),t)

(1, 0, 0)

In [67]:
mut.minors(3)

[1/5981883750*((421*s^2 + 897*s - 145)*(1684*s - 2213) - (421*s^2 + 156*s - 4162)*(842*s + 221))*(496*s^2 - 903*s - 903)*((2*s^4 + 7*s^3 - 4*s + 31)*t + 15) + 1/5981883750*(1125*(s^2 + 3*s + 2)*(s + 1)*t - (496*s^2 + 1422*s + 605)*((2*s^4 + 7*s^3 - 4*s + 31)*t + 15))*((421*s^2 - 1053*s + 72)*(1684*s - 2213) + (3844*s - 3289)*(842*s + 221)) + 1/5981883750*((421*s^2 + 156*s - 4162)*(421*s^2 - 1053*s + 72) + (421*s^2 + 897*s - 145)*(3844*s - 3289))*(2*((2*s^4 + 7*s^3 - 4*s + 31)*t + 15)*(496*s + 523) - 3375*(s + 1)*t),
 -1/2990941875*((421*s^2 + 897*s - 145)*(1684*s - 2213) - (421*s^2 + 156*s - 4162)*(842*s + 221))*((2*s^4 + 7*s^3 - 4*s + 31)*t + 15)*(589*s + 561) - 1/5981883750*(1125*(s^2 + 3*s + 2)*(s + 1)*t - (496*s^2 + 1422*s + 605)*((2*s^4 + 7*s^3 - 4*s + 31)*t + 15))*((1684*s - 2213)*(953*s + 222) - (842*s^2 + 3369*s - 4629)*(842*s + 221)) + 1/5981883750*((842*s^2 + 3369*s - 4629)*(421*s^2 + 897*s - 145) - (421*s^2 + 156*s - 4162)*(953*s + 222))*(2*((2*s^4 + 7*s^3 - 4*s + 31)*t + 15

In [68]:
import time
start_time = time.time()
var('s,t')

P10=vector([t^2 - 1, s*t^2 + t^2, t^2 + 1,1])
mu10=muBasis(P10)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)

mu10

--- 0.03755068778991699 seconds ---


[     -1     1/2   s + 1]
[      0       0      -2]
[      0    -1/2   s + 1]
[t^2 - 1       1       0]

In [69]:
P10

(t^2 - 1, s*t^2 + t^2, t^2 + 1, 1)

In [70]:
(P10*(mu10)).simplify_full()

(0, 0, 0)

In [71]:
[((mu10.minors(len(P10)-1)[i])/P10[len(P10)-i-1]).simplify_full() for i in range(0,len(P10))]

[1, -1, 1, -1]

In [72]:
deg(mu10.column(0),t),deg(mu10.column(1),t),deg(mu10.column(2),t)

(2, 0, 0)

### cone with the rational space curves (r*s,0,s) and (-r*(s^2 - 1)*s/(s^2 + 1), 2*r*s^2/(s^2 + 1), s)

In [73]:
#r=1
import time
start_time = time.time()
var('s,t')

P11=vector([4*s*t^2 - 16*s, 16*s*t, 4*s*t^2 + 16*s, 4*t^2 + 16])
mu11=muBasis(P11)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)


mu11

--- 0.17774271965026855 seconds ---


[   1/8*t     -1/8        0]
[     1/4   1/16*t        0]
[       0        0       -1]
[-1/8*s*t   -1/8*s        s]

In [74]:
(P11*(mu11)).simplify_full()

(0, 0, 0)

In [75]:
[((mu11.minors(len(P11)-1)[i])/P11[len(P11)-i-1]).simplify_full() for i in range(0,len(P11))]

[-1/512, 1/512, -1/512, 1/512]

In [76]:
#r=2
import time
start_time = time.time()
var('s,t')

P12=vector([50*s*t^2 - 1250*s, 500*s*t, 25*s*t^2 + 625*s, 25*t^2 + 625])
mu12=muBasis(P12)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)

mu12

--- 0.12134099006652832 seconds ---


[  1/100*t    -1/100         0]
[     1/20   1/500*t         0]
[        0         0        -1]
[-1/50*s*t   -1/50*s         s]

In [77]:
(P12*(mu12)).simplify_full()

(0, 0, 0)

In [78]:
[((mu12.minors(len(P12)-1)[i])/P12[len(P12)-i-1]).simplify_full() for i in range(0,len(P12))]

[-1/1250000, 1/1250000, -1/1250000, 1/1250000]

### cylinder with the rational space curves (r,0,s) and (-(s^2 - 1)*r/(s^2 + 1), 2*r*s/(s^2 + 1), s)

In [79]:
#r=2
import time
start_time = time.time()
var('s,t')

P13=vector([2*t^2 - 2, 4*t, s*t^2 + s, t^2 + 1])
mu13=muBasis(P13)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)

mu13

--- 0.14236021041870117 seconds ---


[ 1/4*t   -1/4      0]
[   1/4  1/4*t      0]
[     0      0     -1]
[-1/2*t   -1/2      s]

In [80]:
(P13*(mu13)).simplify_full()

(0, 0, 0)

In [81]:
[((mu13.minors(len(P13)-1)[i])/P13[len(P13)-i-1]).simplify_full() for i in range(0,len(P13))]

[-1/16, 1/16, -1/16, 1/16]

In [82]:
#r=1/2
import time
start_time = time.time()
var('s,t')

P14=vector([1/2*t^2 - 1/2, t, s*t^2 + s, t^2 + 1])
mu14=muBasis(P14)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)


mu14

--- 0.16260194778442383 seconds ---


[     t     -1      0]
[     1      t      0]
[     0      0     -1]
[-1/2*t   -1/2      s]

In [83]:
(P14*(mu14)).simplify_full()

(0, 0, 0)

In [84]:
[((mu14.minors(len(P14)-1)[i])/P14[len(P14)-i-1]).simplify_full() for i in range(0,len(P14))]

[-1, 1, -1, 1]

### Degree of t is zero

In [85]:

import time
start_time = time.time()
var('s,t')

P15=vector([s^2, 2+s^3, s^6])
mu15=muBasis(P15)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)


mu15

--- 0.01944756507873535 seconds ---


[s^3 + 2    -2*s]
[   -s^2     s^3]
[      0      -1]

In [86]:
(P15*mu15).simplify_full()

(0, 0)

In [87]:
[((mu15.minors(len(P15)-1)[i])/P15[len(P15)-i-1]).simplify_full() for i in range(0,len(P15))]

[1, -1, 1]

In [88]:

import time
start_time = time.time()
var('s,t')

P16=vector([s^9, 2+s^3, s^6+s^2+7,1])
mu16=muBasis(P16)

time=(time.time() - start_time)
print("--- %s seconds ---" % time)


mu16

--- 0.029876232147216797 seconds ---


[         0         -1          0]
[   s^3 - 2   -s^2 - 7         -1]
[        -1        s^3          0]
[  s^2 + 11 2*s^2 + 14    s^3 + 2]

In [89]:
(P16*mu16).simplify_full()

(0, 0, 0)

In [90]:
[((mu16.minors(len(P16)-1)[i])/P16[len(P16)-i-1]).simplify_full() for i in range(0,len(P16))]

[-1, 1, -1, 1]

In [91]:
import time
start_time = time.time()
var('s,t')
P=vector([s^3+s*t^2+7*s*t-1, 2*s*t+s^2,t, s^2+t^2+1])
mu=muBasis(P)
time=(time.time() - start_time)
print("--- %s seconds ---" % time)

--- 0.227158784866333 seconds ---


In [92]:
mu

[                       -(s - 1)*s*t - t                                   s - 1                                -1/7*s^2]
[                                   -s*t                                       2                            -1/7*s - 1/7]
[(7*s^2 - 3*s)*s*t - (2*s^2 - 7*s)*t - 1                        -7*s^2 + 3*s + t                   s^3 + 2/7*s^2 + 2/7*s]
[                      (s^2 - s + 1)*s*t                            -s^2 + s - 1                                 1/7*s^3]

In [93]:
(P*mu).simplify_full()

(0, 0, 0)

In [94]:
[((mu.minors(len(P)-1)[i])/P[len(P)-i-1]).simplify_full() for i in range(0,len(P))]

[-1/7, 1/7, -1/7, 1/7]